Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Network

In [28]:
batch_size = 128
num_hidden_nodes = 1024
l2_penalty = 0.005
#0.01 # 0.005 # 0.02 #0.03


graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.

    layer_one = tf.matmul(tf_train_dataset, weights1) + biases1
    rel_out = tf.nn.relu(layer_one)  
    logits = tf.matmul(rel_out, weights2) + biases2
    # Loss Function
    l2_component = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_penalty * l2_component
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    ## TODO: this bit.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights1) + biases1), 
                  weights2) + biases2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_test_dataset, weights1) + biases1), 
                  weights2) + biases2)

### Training Run

In [14]:
len(train_labels)
indices = np.random.choice(len(train_labels), batch_size, replace=False)
print(str(indices))

train_labels[indices].shape

[129477 154751  65909   2872 196233  79617 166944  18194 104508 162959
  93215  50702 127634 159973 180116 159193 144281 118847  82700 108673
  12764  94125  58867 113752  78400  13809 161607  13117  85794 169236
 146757 157655   3986  90411 114408 174627 122521  99440  32215 136486
 164848 118034  19786 163815  62129 185864 198635  24557 104264 122543
  93778    837  53061 184014 108307 164022  41111   2320  31426 104564
  94421 138387  81098 198304 153744 167747  70490  74094 158531  95933
  11686 190523  28835 150823  65737 193020  56183  24273  99078   1040
  37142 177954  28088 116207 142306 191470 177256  23975 173228 131441
 159532 140675 164703 146440 102259  54896 195672  41772 150533 150732
   8916  99679 169713  54304  59284 128909 167670  82752  19628  73295
  79180 172337  89295   2529  63394 180259  79217  50987  40262  88856
  74871 140826  50415 121071  31107  84567 121959 102096]


(128, 10)

In [27]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #indices = np.random.choice(len(train_labels), batch_size, replace=False)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    #batch_data = train_dataset[indices]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    #batch_labels = train_labels[indices]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 378.742371
Minibatch accuracy: 14.8%
Validation accuracy: 25.3%
Minibatch loss at step 500: 17.674532
Minibatch accuracy: 82.8%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 8.555744
Minibatch accuracy: 78.9%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 3.392760
Minibatch accuracy: 88.3%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 4.666368
Minibatch accuracy: 85.9%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 2.832988
Minibatch accuracy: 85.2%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 2.596651
Minibatch accuracy: 81.2%
Validation accuracy: 82.9%
Test accuracy: 90.2%


#### Records:

beta = 0 - test acc:90.2%
beta = 0.005 - test acc: 91.4%
beta = 0.01 - test acc: 90.2%
beta = 0.02 - test acc: 89.1%
beta = 0.03 - test acc: 88.4%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [33]:

small_size = 500
small_dataset = train_dataset[:small_size]
small_labels = train_labels[:small_size]

small_labels.shape

(500, 10)

In [35]:
num_steps = 3001

small_size = 200
small_dataset = train_dataset[:small_size]
small_labels = train_labels[:small_size]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_labels.shape[0] - batch_size)
    #indices = np.random.choice(len(train_labels), batch_size, replace=False)
    # Generate a minibatch.
    batch_data = small_dataset[offset:(offset + batch_size), :]
    #batch_data = train_dataset[indices]
    batch_labels = small_labels[offset:(offset + batch_size), :]
    #batch_labels = train_labels[indices]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1866.023926
Minibatch accuracy: 10.9%
Validation accuracy: 22.7%
Minibatch loss at step 500: 128.573273
Minibatch accuracy: 100.0%
Validation accuracy: 67.2%
Minibatch loss at step 1000: 10.551140
Minibatch accuracy: 100.0%
Validation accuracy: 72.0%
Minibatch loss at step 1500: 0.980781
Minibatch accuracy: 100.0%
Validation accuracy: 74.1%
Minibatch loss at step 2000: 0.200384
Minibatch accuracy: 100.0%
Validation accuracy: 74.1%
Minibatch loss at step 2500: 0.133643
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 3000: 0.125923
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Test accuracy: 80.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [41]:
batch_size = 128
num_hidden_nodes = 1024
l2_penalty = 0.005
#0.01 # 0.005 # 0.02 #0.03


graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.

    layer_one = tf.matmul(tf_train_dataset, weights1) + biases1
    rel_out = tf.nn.relu(layer_one)  
    logits = tf.matmul(rel_out, weights2) + biases2
    # Loss Function
    dropped_out = tf.nn.dropout(logits, keep_prob = 0.5)
    softmax_component = tf.nn.softmax_cross_entropy_with_logits(2 * dropped_out, tf_train_labels)
    l2_component = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2)
    loss = tf.reduce_mean(softmax_component + l2_penalty * l2_component)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    ## TODO: this bit.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights1) + biases1), 
                  weights2) + biases2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_test_dataset, weights1) + biases1), 
                  weights2) + biases2)

In [43]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #indices = np.random.choice(len(train_labels), batch_size, replace=False)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    #batch_data = train_dataset[indices]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    #batch_labels = train_labels[indices]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2772.187988
Minibatch accuracy: 13.3%
Validation accuracy: 23.8%
Minibatch loss at step 500: 158.795685
Minibatch accuracy: 75.8%
Validation accuracy: 77.7%
Minibatch loss at step 1000: 12.833202
Minibatch accuracy: 78.9%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 2.270777
Minibatch accuracy: 85.9%
Validation accuracy: 83.1%
Minibatch loss at step 2000: 1.440708
Minibatch accuracy: 82.0%
Validation accuracy: 76.8%
Minibatch loss at step 2500: 1.271687
Minibatch accuracy: 82.8%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 1.315673
Minibatch accuracy: 83.6%
Validation accuracy: 81.8%
Test accuracy: 88.8%


In [42]:
num_steps = 3001

small_size = 200
small_dataset = train_dataset[:small_size]
small_labels = train_labels[:small_size]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_labels.shape[0] - batch_size)
    #indices = np.random.choice(len(train_labels), batch_size, replace=False)
    # Generate a minibatch.
    batch_data = small_dataset[offset:(offset + batch_size), :]
    #batch_data = train_dataset[indices]
    batch_labels = small_labels[offset:(offset + batch_size), :]
    #batch_labels = train_labels[indices]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2521.912598
Minibatch accuracy: 8.6%
Validation accuracy: 13.0%
Minibatch loss at step 500: 141.104767
Minibatch accuracy: 100.0%
Validation accuracy: 67.3%
Minibatch loss at step 1000: 12.288172
Minibatch accuracy: 100.0%
Validation accuracy: 69.9%
Minibatch loss at step 1500: 1.784273
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 2000: 1.038442
Minibatch accuracy: 100.0%
Validation accuracy: 74.3%
Minibatch loss at step 2500: 0.839952
Minibatch accuracy: 100.0%
Validation accuracy: 74.4%
Minibatch loss at step 3000: 0.993242
Minibatch accuracy: 100.0%
Validation accuracy: 74.4%
Test accuracy: 81.7%


In [ ]:
- restricted case with N=200, test accuracy is = 81.7%
- all case with N = 200000, test accuracy is = 88.8%

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [44]:
batch_size = 128
num_hidden_nodes = 1024
l2_penalty = 0.005
starter_learning_rate = 0.5
#0.01 # 0.005 # 0.02 #0.03


graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100000, 0.96, staircase=True)


    
    # Variables.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.

    layer_one = tf.matmul(tf_train_dataset, weights1) + biases1
    rel_out = tf.nn.relu(layer_one)  
    logits = tf.matmul(rel_out, weights2) + biases2
    # Loss Function
    dropped_out = tf.nn.dropout(logits, keep_prob = 0.5)
    softmax_component = tf.nn.softmax_cross_entropy_with_logits(2 * dropped_out, tf_train_labels)
    l2_component = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2)
    loss = tf.reduce_mean(softmax_component + l2_penalty * l2_component)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights1) + biases1), 
                  weights2) + biases2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(
                tf.matmul(tf_test_dataset, weights1) + biases1), 
                  weights2) + biases2)

In [45]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #indices = np.random.choice(len(train_labels), batch_size, replace=False)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    #batch_data = train_dataset[indices]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    #batch_labels = train_labels[indices]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2570.463379
Minibatch accuracy: 14.8%
Validation accuracy: 13.2%
Minibatch loss at step 500: 151.355530
Minibatch accuracy: 75.0%
Validation accuracy: 74.3%
Minibatch loss at step 1000: 13.233480
Minibatch accuracy: 78.9%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 2.235523
Minibatch accuracy: 86.7%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 1.459341
Minibatch accuracy: 89.1%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 1.358265
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 1.305196
Minibatch accuracy: 81.2%
Validation accuracy: 83.3%
Minibatch loss at step 3500: 1.315784
Minibatch accuracy: 82.0%
Validation accuracy: 82.6%
Minibatch loss at step 4000: 1.424379
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Minibatch loss at step 4500: 1.366253
Minibatch accuracy: 86.7%
Validation accuracy: 82.2%
Minibatch loss at step 5000: 1.496435
Minibatch accuracy: 81.2%
Validation a